<h1><b>手書き数字文字の認識</b></h1>
<p>実行内容</p>
<ul>
<li>モジュールのインポート</li>
<li>手書き文字のデータセットの取得</li>
<li>トレーニングデータとテストデータの割り当てと、学習モデル(SVM)の作成</li>
<li>学習モデルのテスト</li>
<ul>

<h3>サポートベクトル分類(SVM)の考え方</h3>
<h5>マージン最大化とサポートベクトル</h5>
<p>サポートベクトルマシンを理解するためには、名前の由来である「サポートベクトル」について理解する必要があります。</p>
<p>サポートベクトルとは、先に説明したように「予測に必要となる一部のデータ」です。<br />
では「予測に必要となる一部のデータ」はどのようにして決められるのでしょうか。<br />
それを定めるのに「マージン最大化」と呼ばれる考え方を使います。</p>

<p>
マージンとは、「判別する境界とデータとの距離」を指します。<br />
これが大きければ、「ほんの少しデータが変わっただけで誤判定してしまう」というミスをなくすことができます。<br />
なお、境界線と最も近くにあるデータを「サポートベクトル」と呼びます。</p>
<p><a onclick="javascript:pageTracker._trackPageview('/downloads/wp-content/uploads/2017/07/svm-7-intro.jpg');"  href="https://logics-of-blue.com/wp-content/uploads/2017/07/svm-7-intro.jpg"><img loading="lazy" src="https://logics-of-blue.com/wp-content/uploads/2017/07/svm-7-intro-1024x576.jpg" alt="" width="640" height="360" class="alignnone size-large wp-image-1803" srcset="https://logics-of-blue.com/wp-content/uploads/2017/07/svm-7-intro-1024x576.jpg 1024w, https://logics-of-blue.com/wp-content/uploads/2017/07/svm-7-intro-300x169.jpg 300w, https://logics-of-blue.com/wp-content/uploads/2017/07/svm-7-intro-768x432.jpg 768w, https://logics-of-blue.com/wp-content/uploads/2017/07/svm-7-intro.jpg 1280w" sizes="(max-width: 640px) 100vw, 640px" /></a></p>


<h2>モジュールのインポート</h2>
<p>このプログラムでは以下のライブラリを使用する。</p>
<ul>
  <li>sklearn <small>　- scikit-learnのライブラリ群</small></li>
  <li>matplotlib <small>　- グラフ等の描画用のライブラリ</small></li>
</ul>

In [ ]:
from sklearn.datasets import load_digits
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt



---



<h2>手書き文字のデータセットの取得</h2>
<p>ここでは、インポートしたデータセットからデータを取得している。<br>実行すると、データセットの中身を確認することができる。</p>

In [ ]:
digits = load_digits()
# digitsデータセットはdir()で調べることができます。
print(dir(digits))
print(digits.data)         # digitデータセットのdataの値を出力
print(digits.data.shape)   # digitデータセットのdataの構造を出力 1797×64の2次元配列 (1797, 64)
print(digits.target.shape) # digitデータセットのtargetの構造を出力　要素が1797個の1次元配列

<p>ここで、データセット内の一つ目のデータを見てみる。中にはピクセルの色の濃度を16進数で表記したデータが、8×8=64ピクセル分の配列として格納されている。<br>出力された画像を見るとデータと画像が対応していることが分かる。<br>(右から左に確認すると、画像の黒さとデータの数値が対応している。)</p>

In [ ]:
# 1行に64列の数字で1文字になる。
print(len((digits.data[0]))) # 64 (64ピクセルというのは、は8×8ピクセルの16階調のグレイスケール画像）
print((digits.data[0]))
print(digits.data[0].shape) # 1次元 (64,)

import numpy as np
# numpyのreshape()を使って8×8に戻す
img = np.reshape(digits.data[0], (8,8))

# cmapにより色を指定可能です。
# cmap=plt.cm.gray_rにより白黒で表現し、interpolationをnearestにすることで、最近傍補間という補完を行なっています。
# 最近傍補間とは、画像を拡大・縮小・回転した際に利用する補間法の1つです。
plt.imshow(img, cmap=plt.cm.gray_r, interpolation='nearest')
# plt.imshow(digits.images[0], cmap=plt.cm.gray_r, interpolation='nearest')

# 上記のcmap=の部分及び、interpolation=を消した下記でも動作します。比べてみてください。
# plt.imshow(image)

plt.axis('off')
plt.show()

次のプログラムを実行すると、このデータセットに含まれているデータの数、10個目のデータセット（とそのラベル）が出力される。
各数字のデータセットごとにそれぞれ特徴が異なることが分かる。

In [ ]:
num = len(digits.data)
print("num=" + str(num)) 

# zip()にて複数リストの値を同時に取得しています。
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:10]):
    plt.subplot(2, 5, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)
plt.show()



---



<h1>トレーニングデータとテストデータの割り当てと、学習モデル(SVM)の作成</h1>
<p>今あるデータを、トレーニングを行うためのデータとテストを行うためのデータをそれぞれ割り振る。トレーニングデータとテストデータに分ける。<br>これは正しく認識出来るかのテストをする画像も残す必要があるので、2/3をトレーニング用の画像として利用する。
<br><br></p>
<p>使用するデータ数を決めたら、トレーニングデータによる学習モデル（SVM）の作成を行う。</p>

In [ ]:
num = len(digits.data)
training_num = int(num*2/3) 
print("training_num=" + str(training_num))
print("training_num_type:" + str(type(training_num)))

#トレーニングデータの取得
train_data = digits.data[:training_num]
train_target = digits.target[:training_num]

#テストデータの取得
test_data = digits.data[training_num:]
test_target = digits.target[training_num:]

# SVMによるモデルを作成します
classifier = svm.SVC(gamma=0.001) # 引数の"gamma"というパラメータは、どれだけ複雑な決定境界を引くかのパラメータになっていますが、現時点で理解する必要はありません。

classifier.fit(train_data, train_target)



---



<h1>学習モデルのテスト</h1>
<p>テストデータをプログラムに処理させる。<br>実行するとPredict(推測)された数字と推測に使われたデータの画像、正解率が表示される。

In [ ]:
predicted = classifier.predict(test_data)

images_and_predictions = list(zip(digits.images[training_num:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:10]):
    plt.subplot(2, 5, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Predic: %i' % prediction)
plt.show()
# 正解率　
metrics.accuracy_score(test_target, predicted )